<center>
<img src="Stone_Soup_Icon_Final_small.png">
<h1>Stone Soup Run Manager Demo 05 - Import Data</h1>
Demonstrating the capabilities of the Stone Soup Run Manager to run batches of experiments.
</center>

Generating Data
----------------
First we'll create some models, which will be used by the tracker components.

This will include a 2D-position constant velocity transition model ($x$, $\dot{x}$, $y$ and $\dot{y}$) generated by combining two 1D models (this allows multiple models to be mixed and generation of *n*-dimension models).

In [1]:
import numpy as np
import datetime
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, ConstantVelocity

transition_model = CombinedLinearGaussianTransitionModel((ConstantVelocity(1), ConstantVelocity(1)))

And a measurement model, which will map the position based detections ($x$ and $y$) to the position in the state.

In [2]:
from stonesoup.models.measurement.linear import LinearGaussian

measurement_model = LinearGaussian(ndim_state=4, mapping=[0, 2], noise_covar=np.diag([10, 10]))

Next we'll create a reader that will ingest sensor data from an external source and format this data as Stone Soup *Detections*.

In [3]:
from stonesoup.reader.aishub import JSON_AISDetectionReader

detections_source = JSON_AISDetectionReader(path='sample_AIS_data.json')

Building Kalman tracker components
------------------------------------

With the detection data ready, we'll now build a Kalman tracker. For this we will need a Kalman predictor, which will utilise the same *transition model* we used in the ground truth simulator.

In [4]:
from stonesoup.predictor.kalman import KalmanPredictor

predictor = KalmanPredictor(transition_model)

And a Kalman updater, utilising the same *measurement model* we used in the detection simulator.

In [5]:
from stonesoup.updater.kalman import KalmanUpdater

updater = KalmanUpdater(measurement_model)

We will also need a data associator to link detections to the "correct" track for the update step: in this case a Nearest Neighbour is fine for this demo. The data associator requires a hypothesiser which calculates some form of score/probability of each track being associated to each detection: in this case using a Mahalanobis distance from track prediction to the detection (which will also generate missed detection hypothesis).

In [6]:
from stonesoup.hypothesiser.distance import DistanceHypothesiser
from stonesoup.measures import Mahalanobis

hypothesiser = DistanceHypothesiser(predictor, updater, Mahalanobis(), missed_distance=3)

In [7]:
from stonesoup.dataassociator.neighbour import NearestNeighbour

data_associator = NearestNeighbour(hypothesiser)

And finally a initiator to generate tracks from unassociated detections, in this case a single point initiator generating a track for every unassociated detection.

In [8]:
from stonesoup.initiator.simple import SinglePointInitiator
from stonesoup.types.state import GaussianState

initiator = SinglePointInitiator(
    GaussianState(np.array([[0], [0], [0], [0]]), np.diag([10000, 100, 10000, 1000])),
    measurement_model=measurement_model)

And a deleter to remove tracks, for this demo simply based on large covariance threshold.

In [9]:
from stonesoup.deleter.error import CovarianceBasedDeleter

deleter = CovarianceBasedDeleter(covar_trace_thresh=1E3)

With all the components in place, we'll now construct the tracker with a multi target tracker.

In [10]:
from stonesoup.tracker.simple import MultiTargetTracker

tracker = MultiTargetTracker(
    initiator=initiator,
    deleter=deleter,
    detector=detections_source,
    data_associator=data_associator,
    updater=updater,
)

Creating the Metrics package
----------------------------
We now construct the Metrics package that will be run against the output of the multi target tracker, encoded in the format that the Run Manager understands. NOTE: Since we imported data as *Detections* and there is no *Groundtruth*, and most of these metrics require *Groundtruth*, most of these metrics will return empty values.

In [12]:
from stonesoup.metricgenerator import BasicMetrics
from stonesoup.metricgenerator.ospametric import OSPAMetric
from stonesoup.metricgenerator.tracktotruthmetrics import SIAPMetrics
from stonesoup.dataassociator.tracktotrack import EuclideanTrackToTruth

basic_calculator = BasicMetrics()
ospa_calculator = OSPAMetric(c=10, p=1, measurement_model_track=measurement_model,
                             measurement_model_truth=measurement_model)
siap_calculator = SIAPMetrics()

associator = EuclideanTrackToTruth(measurement_model_truth=measurement_model, measurement_model_track=measurement_model,
                                   association_threshold=30)

metrics_conditions = {'metric_01': basic_calculator, 'metric_02': ospa_calculator, 
                      'metric_03': siap_calculator, 'associator':associator}

Assembling the Run Manager Experiment
-------------------------------------
We now assemble to components of the Run Manager experiment, encoded in the format that the Run Manager understands.

In [13]:
config_top_level = {}
config_top_level['tracker01'] = tracker
config_top_level['components'] = {}
config_top_level['conditions'] = {}
config_top_level['metrics'] = metrics_conditions

Running the Run Manager Experiment
----------------------------------
We then run the experiment and output

In [14]:
from stonesoup.runmanager.manager import SimpleRunManager

local_run_manager = SimpleRunManager(run_checks=True)
local_run_manager.load_config(object=config_top_level)

local_run_manager.run_experiment()
local_run_manager.output_experiment_results()

Tracker processed.


The results have been output to the following text file:

In [15]:
print(local_run_manager.experiment_results_filename)

run_manager_results\2019-06-10_23-29_experiment_results.txt
